In [ ]:
# Import Required Libraries
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
#Used headers/agent as the request timed out and asking for agent. Using following code you can fake the agent.
headers={'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}

In [ ]:
# Target url
url="https://www.swiggy.com"

# Content from swiggy homepage and saving all the city name listed there
response_city = requests.get(url,headers=headers)
content_city = response_city.content
soup_city = BeautifulSoup(content_city,"html.parser")
name_city =soup_city.find_all("div",attrs={"class": "_2Im4A"})

In [ ]:
# Saving the links of all city on swiggy website

%%time
url_city=[]
for i in range(3,7):
    for a in name_city[i].find_all('a', href=True):
            url_city.append(a['href'])
print("Total City: ",len(url_city))

Swiggy has listed restaurants in two ways.
First, if the city is small it has listed all the restaurant as it is.
Second, if the city is big then it is divided into region wise and all the restaurant are listed according to there region.
So following code will go to each city page and check if has been divided into region or not. If yes, crawler will store links of all region otherwise it will store links of all restaurant.

In [ ]:
# Saving all the links of region where city is divided into region
# Saving all the links of the restaurants in a city
%%time
url_region=[]
url_rest1=[]
for i in range(len(url_city)):
    try:
        response_region = requests.get(url+url_city[i],headers=headers)
        time.sleep(5)
        if response_region.status_code==200:
            content_region = response_region.content
            soup_region = BeautifulSoup(content_region,"html.parser")
            name_region =soup_region.find_all("div",attrs={"class": "zIQo_"})
            if not name_region:
                page= soup_region.find_all("div",attrs={"class": "_2OmLw"})
                if not page:
                    pass
                else:            
                    u=[]
                    for a in page[0].find_all('a', href=True):
                        if(a['href']=='#'):
                            u.append(url+url_city[i])
                        else:
                            u.append(url+a['href'])
                    for j in range(len(u)):
                        response = requests.get(u[j],headers=headers)
                        content = response.content
                        soup = BeautifulSoup(content,"html.parser")
                        link=soup.find_all("div",attrs={"class": "_3XX_A"})
                        for i in range(len(link)):
                            for a in link[i].find_all('a', href=True):
                                    url_rest1.append(a['href'])
            else:
                for l in range(len(name_region)):
                    for a in name_region[l].find_all('a', href=True):
                        url_region.append(a['href'])
    except Exception as ex:
        print(str(ex))
print("Total Area: ", len(url_region))
print(len(url_rest1))

In [ ]:
len(url_rest1)

In [ ]:
# Saving all the links of the resturant in a city

%%time
url_rest2=[]
for i in range(len(url_region)):
    try:
        response_page = requests.get(url+url_region[i],headers=headers)
        time.sleep(5)
        if response_pag e.status_code==200:
            content_page = response_page.content
            soup_page = BeautifulSoup(content_page,"html.parser")
            page= soup_page.find_all("div",attrs={"class": "_1MNQO"})
            if page:
                u=[]
                for a in page[0].find_all('a', href=True):
                    u.append(a['href'])
                for j in range(len(u)):
                    response = requests.get(url+u[j],headers=headers)
                    content = response.content
                    soup = BeautifulSoup(content,"html.parser")
                    data = json.loads(soup.find('script', type='application/ld+json').text)
                    dic=data['itemListElement']
                    for k in range(len(dic)):
                        a=dic[k]['url']
                        url_rest2.append(a)
    except Exception as ex:
        print(str(ex))
print(len(url_rest2))

In [ ]:
# Total urls
url_rest=url_rest1+url_rest2

In [ ]:
# Converting urls into dataframe
links=pd.DataFrame()
links['links']=url_rest
links.to_csv("links.csv",index=False)